<a href="https://colab.research.google.com/github/itssud/MovieTicketing-Backend/blob/main/Movie_Ticketing_System_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import datetime

In [ ]:
# Enter number of screens
#no_of_screens = int(input('Enter the Number of Screens in the Theatre : '))
no_of_screens = 2

screens = ['AUDI' + str(screen+1) for screen in range(no_of_screens) ]

print(f'Available Screens are : {screens}')

movies = ['Spiderman', 'Dark Knight', 'Iron Man', 'Captain America', 'Minal Murali']

show_timings = {'early morning' : '8:00 AM', 'morning' : '11:00 AM', 'matniee' : '1:40 PM', 'afternoon' : '4:25 PM', 'evening': '7:10 PM', 'night': '10:30 PM'}

no_of_seats = {screens[id] : random.randrange(50, 220) for id in range(no_of_screens)}

row_letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

seat_categories_price = {'Elite' : 64, 'Economy' : 120, 'Premium' : 170, 'Super Premium' : 195}

Available Screens are : ['AUDI1', 'AUDI2']


In [ ]:
def getRowsAndCols(screen):
  while True:
      col = random.randrange(5,25)
      row = no_of_seats[screen] // col
      remaining = no_of_seats[screen] % col
  
      if row > 8 and row < len(row_letters)-7 and remaining < col:
        return (row,col, remaining)

getRowsAndCols('AUDI1')

(11, 13, 9)

In [ ]:
def setShows(screen):
  shows = {}
  get_row_col_data = getRowsAndCols(screen)
  row = get_row_col_data[0]
  col = get_row_col_data[1]
  remaining = get_row_col_data[2]

  for show in list(show_timings.keys()):

    rows = { row_letters[idx] : [ num+1 for num in range(col) ] for idx in range(row) }
    if remaining:
        rows[row_letters[len(rows)]] = [n+1 for n in range(remaining)]
    
    shows[show] = rows
  
  return shows


In [ ]:
shows = {}
running_movies = {}

for screen in screens:
  shows[screen] = setShows(screen)
  running_movies[screen] = { showtime : random.choice(movies) for showtime in show_timings }

In [ ]:
def GetRowCategories(screen):

    seats_of_audi = shows[screen]['early morning']
    
    seat_categories = {}
    
    count = 0
    super_premium = []
    premium = []
    elite = []
    economy = []

    for key in seats_of_audi.keys():
        
        if count == 0:
            super_premium.append(key)
        elif count > 0 and count < 3:
            premium.append(key)
        elif count > len(seats_of_audi)-3 and count <= len(seats_of_audi):
            elite.append(key)
        else:
            economy.append(key)
            
        seat_categories['Super Premium'] = super_premium
        seat_categories['Premium'] = premium
        seat_categories['Economy'] = economy
        seat_categories['Elite'] = elite
        count+=1
    
    return seat_categories

In [ ]:
seat_categories = {}
for screen in screens:
  seat_categories[screen] = GetRowCategories(screen)

In [ ]:
def getMovieTimings(movie):
  if movie in movies:
    showlist = {}
    for screen in screens:
      showtimes = []
      for showtime, running_movie in running_movies[screen].items():
        if running_movie == movie:
          showtimes.append(showtime)
      showlist[screen] = showtimes
    return showlist
  else: return None

In [ ]:
def getTotalandRowAvailableSeats(screen, showtime, category):
  rows = seat_categories[screen][category]
  row_available_seats = {}
  total_available = 0
  for row in rows:
    available_seat_count = 0
    col = shows[screen][showtime][row]
    for seat in col:
      if isinstance(seat, int):
        available_seat_count += 1
        total_available += 1
    row_available_seats[row] = available_seat_count

  return (total_available, row_available_seats)

In [ ]:
bookingData = []

def bookTicket():
  print('Welcome to the Cinemas!!!')
  name = input('Enter your Name to proceed... ')

  print(f'Hi {name}, Select the desired movie to watch from the following...')
  for movie_idx in range(len(movies)):
    print(f'[ {movie_idx} ]   {movies[movie_idx]}')
  movie_selector = int(input('Enter the number assosiated with the movie you wanna watch... '))
  while movie_selector > len(movies)-1 or movie_selector < 0:  
    movie_selector = int(input('[Not Found] Please enter a valid number... '))
  movie = movies[movie_selector]
  print(f'\nHey {name}, you have selected {movie} to watch, Please select the preferred showtime to watch from the following...')
  movie_timings = getMovieTimings(movie)

  screen_time_pair = []
  for screen, showtimes in movie_timings.items():
    for showtime in showtimes:
      screen_time_pair.append([screen, showtime])

  for pair_idx in range(len(screen_time_pair)):
    pair = screen_time_pair[pair_idx]
    print(f'[ {pair_idx} ]    Screen : {pair[0]} - ShowTime : {pair[1]}  ( { pair[1] } )')

  pair_selector = int(input('Enter the number assosiated with the showtiming you prefer... '))
  while pair_selector > len(screen_time_pair)-1 or pair_selector < 0:   pair_selector = int(input('[Not Found] Please enter a valid number... '))

  pair_screen = screen_time_pair[pair_selector][0]
  pair_showtime = screen_time_pair[pair_selector][1]
  print(f'\nAwesome {name}, you have selected the Movie {movie} on {pair_screen} screen at {pair_showtime} ( {show_timings[pair_showtime]} ) ')
  print('Further select the category of seats you prefer from the below list : ')

  categories = list(seat_categories_price.keys())

  available_rows = {}
  for category_idx in range(len(categories)):
    get_row_seats_available = getTotalandRowAvailableSeats(pair_screen, pair_showtime, categories[category_idx])
    row_seats_available = get_row_seats_available[0]
    available_rows[category_idx] = get_row_seats_available[1]
    print(f'[ {category_idx} ]    {categories[category_idx]} - Price : {seat_categories_price[ categories[category_idx] ]} INR / ticket  -   Available : {row_seats_available} Seats')

  category_selector = int(input('Enter the number assosiated with the category of seat you want... '))
  while category_selector > len(categories)-1 or category_selector < 0:   category_selector = int(input('[Not Found] Please enter a valid number... '))
  
  print(f'\n{name}, you have selected {categories[category_selector]} seat category.')
  qty = int(input('Enter the quantity of tickets you want... '))

  seat_qty_available = getTotalandRowAvailableSeats(pair_screen, pair_showtime, categories[category_selector])[0]
  while qty > seat_qty_available or qty > 10:  
    if qty > 10:   qty = int(input('Sorry, user cannot book more than 10 tickets at a instance, Please enter a lower quantity... '))
    else:   qty = int(input('Sorry, we dont have enough seats in this category, Please enter a lower quantity... '))
  
  count = 0
  tickets = []
  for rows in available_rows[category_selector]:
    row = shows[pair_screen][pair_showtime][rows]
    for i in range(len(row)):
      if isinstance(row[i], int):
        if count <= qty-1:
          tickets.append( str(rows) + str(row[i]) )
          shows[pair_screen][pair_showtime][rows][i] = name
          count += 1

  
  print(f'[ Booking Date & Time : {datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")} ]\n{name}, you are booking {qty} tickets for {movie} on {pair_screen} at {pair_showtime} on {show_timings[pair_showtime]}, \nSelected tickets are... { ", ".join(tickets)}')
  price_per_ticket = seat_categories_price[categories[category_selector]]
  price = price_per_ticket * qty
  gst = round(price * 0.18, 1)
  payment_check = input(f'''Booking {qty} tickets will cost...
  Base Amount ( INR { price_per_ticket } X {qty} Nos )    =     INR {price}
  Integrated GST (IGST) @ 18%                             =     INR {gst}
  Total Amount Payable                                    =     INR {price + gst}
  Proceed for payment ( Y / N )... ''')

  dateandtime = datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")
  if payment_check.lower() == 'y' or payment_check.lower() == 'yes':
      print(f'[ Payment Confirmed ][ Payment Date & Time : {dateandtime} ]\n{name}, you have booked {qty} tickets for {movie} on {pair_screen} at {pair_showtime} on {show_timings[pair_showtime]}, \Booked tickets are... { ", ".join(tickets)}')
      bookingData.append((name, movie, pair_screen, pair_showtime, tickets, price + gst ,dateandtime, 'Booking'))
  else: print('[ Payment Cancelled ] Please try again....')


In [ ]:
bookTicket()

Welcome to the Cinemas!!!
Enter your Name to proceed... sri
Hi sri, Select the desired movie to watch from the following...
[ 0 ]   Spiderman
[ 1 ]   Dark Knight
[ 2 ]   Iron Man
[ 3 ]   Captain America
[ 4 ]   Minal Murali
Enter the number assosiated with the movie you wanna watch... 4

Hey sri, you have selected Minal Murali to watch, Please select the preferred showtime to watch from the following...
[ 0 ]    Screen : AUDI1 - ShowTime : matniee  ( matniee )
[ 1 ]    Screen : AUDI1 - ShowTime : afternoon  ( afternoon )
[ 2 ]    Screen : AUDI2 - ShowTime : afternoon  ( afternoon )
[ 3 ]    Screen : AUDI2 - ShowTime : night  ( night )
Enter the number assosiated with the showtiming you prefer... 2

Awesome sri, you have selected the Movie Minal Murali on AUDI2 screen at afternoon ( 4:25 PM ) 
Further select the category of seats you prefer from the below list : 
[ 0 ]    Elite - Price : 64 INR / ticket  -   Available : 12 Seats
[ 1 ]    Economy - Price : 120 INR / ticket  -   Available

In [ ]:
bookTicket()

Welcome to the Cinemas!!!
Enter your Name to proceed... sri
Hi sri, Select the desired movie to watch from the following...
[ 0 ]   Spiderman
[ 1 ]   Dark Knight
[ 2 ]   Iron Man
[ 3 ]   Captain America
[ 4 ]   Minal Murali
Enter the number assosiated with the movie you wanna watch... 4

Hey sri, you have selected Minal Murali to watch, Please select the preferred showtime to watch from the following...
[ 0 ]    Screen : AUDI1 - ShowTime : matniee  ( matniee )
[ 1 ]    Screen : AUDI1 - ShowTime : afternoon  ( afternoon )
[ 2 ]    Screen : AUDI2 - ShowTime : afternoon  ( afternoon )
[ 3 ]    Screen : AUDI2 - ShowTime : night  ( night )
Enter the number assosiated with the showtiming you prefer... 2

Awesome sri, you have selected the Movie Minal Murali on AUDI2 screen at afternoon ( 4:25 PM ) 
Further select the category of seats you prefer from the below list : 
[ 0 ]    Elite - Price : 64 INR / ticket  -   Available : 12 Seats
[ 1 ]    Economy - Price : 120 INR / ticket  -   Available

In [ ]:
def cancelTicket(name):
  user_data_index = []
  for idx in range(len(bookingData)):
    data = bookingData[idx]
    if data[0] == name and data[7] == 'Booking':
      print(f'[ {idx} ] {data[6]} : {data[1]} -> { data[2] } @ {data[3]} | {data[4]} | INR {data[5]} | Booked By : {data[0]}')
      user_data_index.append(idx)
  user_data_selector = int(input('Select the appropriate ticket which is to be cancelled... '))
  while user_data_selector not in user_data_index:  user_data_selector = int(input('[ Invalid Selection ] Try again with the appropriate ticket which is to be cancelled... '))

  datapoint = bookingData[user_data_selector]
  cancel_ticket = input(f'''You have selected {datapoint[6]} : {datapoint[1]} -> { datapoint[2] } @ {datapoint[3]} ( {show_timings[datapoint[3]]} ) | INR {datapoint[5]} | Booked By : {datapoint[0]}
  {datapoint[4]}
  Type the ticket number / numbers which are to be cancelled... ''')

  datapoint_per_price = len(datapoint[4]) / datapoint[5]
  available_for_cancellation = []
  not_found = []
  tickets = [ tkt.strip() for tkt in cancel_ticket.split(',') ]
  for idx in range(len(tickets)):
    if tickets[idx] in datapoint[4]:
      available_for_cancellation.append(tickets[idx])
    else:
      not_found.append(tickets[idx])

  ticket_checker = input(f'''{', '.join(available_for_cancellation)} are considered for cancellation, excluding invalid entries... Do you want to continue cancelling { ', '.join(available_for_cancellation) }?? ( Y / N )... ''')
  if ticket_checker.lower() == 'y' or ticket_checker.lower() == 'yes':
    for tkt_num in available_for_cancellation:
      shows[datapoint[2]][datapoint[3]][tkt_num[:1]][int(tkt_num[1:]) - 1] = int(tkt_num[1:])
      bookingData[user_data_selector][4].remove(tkt_num)
    
  bookingData.append( (datapoint[0], datapoint[1], datapoint[2], datapoint[3], available_for_cancellation, len(available_for_cancellation) * datapoint_per_price, datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S"), 'Cancellation') )


In [ ]:
bookingData

[('sri',
  'Minal Murali',
  'AUDI2',
  'afternoon',
  ['D1', 'D2', 'D3', 'D4', 'D5'],
  708.0,
  '31/01/2022 04:52:17',
  'Booking'),
 ('sri',
  'Minal Murali',
  'AUDI2',
  'afternoon',
  ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9'],
  2070.9,
  '31/01/2022 04:54:02',
  'Booking')]

In [ ]:
cancelTicket('sri')

[ 0 ] 31/01/2022 04:52:17 : Minal Murali -> AUDI2 @ afternoon | ['D1', 'D2', 'D3', 'D4', 'D5'] | INR 708.0 | Booked By : sri
[ 1 ] 31/01/2022 04:54:02 : Minal Murali -> AUDI2 @ afternoon | ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9'] | INR 2070.9 | Booked By : sri
Select the appropriate ticket which is to be cancelled... 0
You have selected 31/01/2022 04:52:17 : Minal Murali -> AUDI2 @ afternoon ( 4:25 PM ) | INR 708.0 | Booked By : sri
  ['D1', 'D2', 'D3', 'D4', 'D5']
  Type the ticket number / numbers which are to be cancelled... D1
